# Análise de Dados com Python - Portal da transparência

In [ ]:
caminho_dados = 'https://drive.google.com/uc?export=download&id=1nqEvkBuIsNxCjrrv2pr9zyZajNIPVR3Q'

In [ ]:
import pandas as pd 

pd.set_option('display.max_columns', None)
df_viagens = pd.read_csv(caminho_dados, encoding='W')
